In [1]:
import numpy as np
import pandas as pd
import string
import re

In [2]:
with open('E:/FILE of Trong/NLP Project/ner/improve_model_v2/businessCard.txt', mode= 'r', encoding='utf-16', errors='ignore') as f:
    text = f.read()

In [3]:
data = list(map(lambda x: x.split('\t'), text.split('\n')))

In [4]:
df = pd.DataFrame(data[1:], columns = data[0])

In [5]:
df.head()

,id,text,tag
0,000.jpeg,,O
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE


## Cleaning Text
- Remove white space
- Remove Unwanted specical characters

In [6]:
string.whitespace

' \t\n\r\x0b\x0c'

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tableWhitespace = str.maketrans('', '', whitespace)
tablePunctuation = str.maketrans('','', punctuation)
def cleanText(txt):
    text = str(txt)
    # text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    return str(removepunctuation)

In [9]:
df['text'] = df['text'].apply(cleanText)

In [10]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace = True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9224\1371438719.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace = True)


In [11]:
dataClean.head(10)

,id,text,tag
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
6,000.jpeg,Fi,O
7,000.jpeg,/laurelsoverseaseducation,O
8,000.jpeg,‰ÛÏ@,O
9,000.jpeg,LAURELS,B-ORG
10,000.jpeg,OVERSEAS,I-ORG


# Convert Data into Spacy Format

In [12]:
group = dataClean.groupby(by='id')

In [13]:
cards = group.groups.keys()

In [14]:
allCardsData = []
for card in cards:
    cardData = []
    grouparray = group.get_group(card)[['text','tag']].values
    content = ''
    annotations = {'entities': []}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1
        start = end
        end = start + stringLength
        if label != 'O':
            annot = (start, end-1, label)
            annotations['entities'].append(annot)
        content = content + text + ' '
    cardData = (content, annotations)
    allCardsData.append(cardData)

In [15]:
# allCardsData

In [15]:
card_data_df = pd.DataFrame(allCardsData, columns = ['text', 'labels'])

In [16]:
card_data_df['isNull'] = card_data_df['labels'].apply(lambda x: 'Null' if len(x['entities']) == 0 else 'Clean')

### Null entries need to drop

In [17]:
card_data_df.query('isNull == "Null"')

,text,labels,isNull
27,ae Be 3,{'entities': []},Null
60,"Le ""95,"" Nungambakkam High ""Road,"" ""Nungambakk...",{'entities': []},Null


In [18]:
card_data_df.dropna(inplace=True)
clean_data = card_data_df.query('isNull == "Clean"')[['text','labels']]

In [19]:
allCardsData = list(map(lambda x: tuple(x), clean_data.values.tolist()))

In [20]:
# allCardsData

### Split Data into Traning and Testing

In [21]:
import random

In [22]:
random.shuffle(allCardsData)

In [23]:
len(allCardsData)

265

In [24]:
TrainData = allCardsData[:240]
TestData = allCardsData[240:]

In [25]:
import pickle

In [27]:
pickle.dump(TrainData, open('E:/FILE of Trong/NLP Project/ner/improve_model_v2/data/TrainData.pickle', mode='wb'))
pickle.dump(TestData, open('E:/FILE of Trong/NLP Project/ner/improve_model_v2/data/TestData.pickle', mode='wb'))